In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical

In [2]:
df = pd.read_csv("/kaggle/input/human-disease-prediction-dataset/Training.csv")  # Replace with your actual file name
test_data = pd.read_csv("/kaggle/input/human-disease-prediction-dataset/Testing.csv")    # Replace with your actual file name

# Separate features and target for train and test sets
X_train = train_data.iloc[:, 1:-1]
y_train = pd.get_dummies(train_data['prognosis'])  # One-hot encode the target
X_test = test_data.iloc[:, 1:-1]
y_test = pd.get_dummies(test_data['prognosis'])

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/human-disease-prediction-dataset/Training.csv'

In [ ]:
# Step 2: Separate features (symptoms) and target (prognosis)
X = df.iloc[:, 1:-1]  # All symptom columns (assumes first column is an ID or disease name)
y = df["prognosis"]   # Target column (disease names)


In [ ]:
X.head(2)

In [ ]:
y.head(3)

In [ ]:
y.shape

In [ ]:
X_train =X
y_train =y

In [ ]:
X_train.shape

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

In [ ]:
y_train.shape

In [ ]:
# Step 2: Encode the target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_train_categorical = to_categorical(y_train_encoded)

In [ ]:
# Save label classes for later use in Streamlit
np.save("label_classes.npy", label_encoder.classes_)

In [ ]:
# Step 3: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
# Save the scaler for Streamlit
joblib.dump(scaler, "scaler.pkl")

In [ ]:
X_train_scaled.shape

In [ ]:
X_train_scaled.shape[1]

In [ ]:
y_train_categorical.shape[1]

In [ ]:
y_train_categorical.shape

In [ ]:
# Build the ANN model
model = Sequential([
    Dense(256, activation='relu', input_dim=X_train_scaled.shape[1]),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_train_categorical.shape[1], activation='softmax')
])

In [ ]:
# Step 5: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 6: Train the model
history = model.fit(X_train_scaled, y_train_categorical, epochs=50, batch_size=32, validation_split=0.2)

In [ ]:
# Step 7: Save the model and encoder
model.save("disease_prediction_ann_model.h5")

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the output of model.fit()

# Plot Training and Validation Accuracy
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='x')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

# Plot Training and Validation Loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='x')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


In [ ]:
test_data = pd.read_csv("/kaggle/input/human-disease-prediction-dataset/Testing.csv")

In [ ]:
X_test = test_data.iloc[:, 1:-1]
y_test = test_data["prognosis"] 

In [ ]:
X_test.head(2)

In [ ]:
y_test.shape

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_test.shape

In [ ]:
# Encode the test labels
y_test_encoded = label_encoder.transform(y_test)
y_test_categorical = to_categorical(y_test_encoded)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Save the symptom order for Streamlit
symptom_names = X_train.columns.tolist()
with open("symptom_order.txt", "w") as file:
    file.write("\n".join(symptom_names)) 